# Submission Template

So for submission this is how it is going to look:


<code>ID Width Height Predicition_String</code>

Where Prediction string is:

<code>0_class cell_0_class_prob cell_0_rle_binary_mask_encoded
1_class cell_0_class_prob cell_0_rle_binary_mask_encoded
2_class cell_0_class_prob cell_0_rle_binary_mask_encoded    
3_class cell_0_class_prob cell_0_rle_binary_mask_encoded
4_class cell_0_class_prob cell_0_rle_binary_mask_encoded
5_class cell_0_class_prob cell_0_rle_binary_mask_encoded
6_class cell_0_class_prob cell_0_rle_binary_mask_encoded    
7_class cell_0_class_prob cell_0_rle_binary_mask_encoded     
8_class cell_0_class_prob cell_0_rle_binary_mask_encoded 
9_class cell_0_class_prob cell_0_rle_binary_mask_encoded 
10_class cell_0_class_prob cell_0_rle_binary_mask_encoded     
11_class cell_0_class_prob cell_0_rle_binary_mask_encoded    
12_class cell_0_class_prob cell_0_rle_binary_mask_encoded     
13_class cell_0_class_prob cell_0_rle_binary_mask_encoded     
14_class cell_0_class_prob cell_0_rle_binary_mask_encoded     
15_class cell_0_class_prob cell_0_rle_binary_mask_encoded     
16_class cell_0_class_prob cell_0_rle_binary_mask_encoded 
17_class cell_0_class_prob cell_0_rle_binary_mask_encoded     
18_class cell_0_class_prob cell_0_rle_binary_mask_encoded</code>     
    
<code>0_class cell_1_class_prob cell_1_rle_binary_mask_encoded
1_class cell_1_class_prob cell_1_rle_binary_mask_encoded
2_class cell_1_class_prob cell_1_rle_binary_mask_encoded    
3_class cell_1_class_prob cell_1_rle_binary_mask_encoded
4_class cell_1_class_prob cell_1_rle_binary_mask_encoded
5_class cell_1_class_prob cell_1_rle_binary_mask_encoded
6_class cell_1_class_prob cell_1_rle_binary_mask_encoded    
7_class cell_1_class_prob cell_1_rle_binary_mask_encoded     
8_class cell_1_class_prob cell_1_rle_binary_mask_encoded 
9_class cell_1_class_prob cell_1_rle_binary_mask_encoded 
10_class cell_1_class_prob cell_1_rle_binary_mask_encoded     
11_class cell_1_class_prob cell_1_rle_binary_mask_encoded    
12_class cell_1_class_prob cell_1_rle_binary_mask_encoded     
13_class cell_1_class_prob cell_1_rle_binary_mask_encoded     
14_class cell_1_class_prob cell_1_rle_binary_mask_encoded     
15_class cell_1_class_prob cell_1_rle_binary_mask_encoded     
16_class cell_1_class_prob cell_1_rle_binary_mask_encoded 
17_class cell_1_class_prob cell_1_rle_binary_mask_encoded     
18_class cell_1_class_prob cell_1_rle_binary_mask_encoded</code>         
    
    
...

<code>0_class cell_n_class_prob cell_n_rle_binary_mask_encoded
1_class cell_n_class_prob cell_n_rle_binary_mask_encoded
2_class cell_n_class_prob cell_n_rle_binary_mask_encoded    
3_class cell_n_class_prob cell_n_rle_binary_mask_encoded
4_class cell_n_class_prob cell_n_rle_binary_mask_encoded
5_class cell_n_class_prob cell_n_rle_binary_mask_encoded
6_class cell_n_class_prob cell_n_rle_binary_mask_encoded    
7_class cell_n_class_prob cell_n_rle_binary_mask_encoded     
8_class cell_n_class_prob cell_n_rle_binary_mask_encoded 
9_class cell_n_class_prob cell_n_rle_binary_mask_encoded 
10_class cell_n_class_prob cell_n_rle_binary_mask_encoded     
11_class cell_n_class_prob cell_n_rle_binary_mask_encoded    
12_class cell_n_class_prob cell_n_rle_binary_mask_encoded     
13_class cell_n_class_prob cell_n_rle_binary_mask_encoded     
14_class cell_n_class_prob cell_n_rle_binary_mask_encoded     
15_class cell_n_class_prob cell_n_rle_binary_mask_encoded     
16_class cell_n_class_prob cell_n_rle_binary_mask_encoded 
17_class cell_n_class_prob cell_n_rle_binary_mask_encoded     
18_class cell_n_class_prob cell_n_rle_binary_mask_encoded</code>     
    
    
    
So as you can see, It is going to be pretty long.

The sample_submission.csv contains all the test folder ID's so all we need to do is predict on all of those images in the above format.

In [6]:
tez_path = '../input/dyno-pytorch'
import sys
sys.path.append(tez_path)

In [7]:
# For pixel normalisation
MEAN_CHANNEL_VALUES=(0.485, 0.456, 0.406)
CHANNEL_STD_DEV=(0.229, 0.224, 0.225)
#MEAN_CHANNEL_VALUES = (0.07730, 0.05958, 0.07135)  # RGB
#CHANNEL_STD_DEV = (0.12032, 0.08593, 0.14364)

# Path to weights
MODEL_PATH = '../input/models/model_split_0.bin'
RESNET_PATH = '../input/models/resnet18.pth'

In [8]:
!pip install -q "../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl"
!pip install -q "../input/hpapytorchzoozip/pytorch_zoo-master"
!pip install -q "../input/hpacellsegmentatormaster/HPA-Cell-Segmentation-master"

In [17]:
# imports
import pandas as pd 
import base64
import numpy as np
from pycocotools import _mask as coco_mask
from typing import List, Tuple
import zlib

import imageio
import matplotlib.pyplot as plt
import random

# model
import torch
from torch import nn
from torchvision.models import resnet18
import tez
import albumentations as A
import numpy as np
from skimage.transform import resize

# HPA
import os
import imageio
from hpacellseg.cellsegmentator import CellSegmentator
from hpacellseg.utils import label_cell

In [10]:
import hpacellseg.cellsegmentator as cellsegmentator
from hpacellseg.utils import label_cell, label_nuclei

NUC_MODEL = '../input/hpacellsegmentatormodelweights/dpn_unet_nuclei_v1.pth'
CELL_MODEL = '../input/hpacellsegmentatormodelweights/dpn_unet_cell_3ch_v1.pth'

segmentator = cellsegmentator.CellSegmentator(
    NUC_MODEL,
    CELL_MODEL,
    scale_factor=0.25,
    device='cuda',
    padding=False,
    multi_channel_model=True
)

please compile abn


/opt/conda/lib/python3.7/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'pytorch_zoo.unet.DPNUnet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set

# assigning bio style variables to images

<code>mt = red_images # microtubules
er = yellow_images # endoplasmic reticulum
nu = blue_images # nucleus 
pr = green_images # [proteins]
images = [mt, er, nu]</code>

## Model

In [11]:
class ResNet18(tez.Model):
    '''Model class to facilitate transfer learning 
    from a resnet-18 model
    '''
    NUM_CLASSES = 19
    DROPOUT_RATE = 0.1
    
    def __init__(self, resnet, train_df=None, valid_df=None, batch_size=16, train_aug=None, valid_aug=None, pretrained=True):
        # Initialise pretrained net and final layers for cell classification
        super().__init__()
        self.convolutions = nn.Sequential(*(list(resnet.children())[0:-1]))
        self.dropout = nn.Dropout(self.DROPOUT_RATE)
        self.dense = nn.Linear(512, self.NUM_CLASSES)
        self.out = nn.Sigmoid()
        self.loss_fn = nn.BCELoss()
                
    def forward(self, image, target=None):
        batch_size = image.shape[0]
        
        # Extracts 512x1 feature vector from pretrained resnet18 conv layers
        x = self.convolutions(image).reshape(batch_size, -1)
        # Fully connected dense layer to 19 class output
        output = self.dense(self.dropout(x))
        # Sigmoid activations on output to infer class probabilities
        output_probs = self.out(output)
        
        if target is not None:
            loss = self.loss_fn(output_probs, target.to(torch.float32))  # why to float32???
            metrics = self.monitor_metrics(output_probs, target)
            return output_probs, loss, metrics
        return output_probs, None, None
    
    def fetch_optimizer(self):
        opt = torch.optim.Adam(self.parameters(), lr=3e-4)
        return opt
    
    def fetch_scheduler(self):
        sch = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            self.optimizer, T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1
        )
        return sch

inf_aug = A.Compose([
    A.Normalize(
        mean=(0.485, 0.456, 0.406), 
        std=(0.229, 0.224, 0.225), 
        max_pixel_value=1.0,
        p=1.0
    )
])    

In [12]:
def infer(img, model, inf_aug):
    ''' In: np.array (512, 512, 3) image
        Out: np.array (1, 19) label-probs
    '''
    # Augment; re-shuffle channels; reshape; send to gpu
    X = inf_aug(image=img)['image']
    X = np.transpose(img, (2, 0, 1)).astype(np.float32)
    X = torch.tensor(X, dtype=torch.float32).to('cuda')
    X = X.unsqueeze(0)
    with torch.no_grad():
        out = model(X)[0]
    return out.cpu().detach().numpy()


## Helper Functions

In [13]:
def get_inference(rgby_channel: tuple, model, inf_aug) -> tuple:
    """
    Input: rgby_channel for cell
    Pass: pass on model and augmentations for inference
    Output: prediction from inference of model
    
    NB: This function currently randomly predicts each class
    """
    # unpack rgby channel
    r, g, b, y = rgby_channel
    # stack img array
    img = np.dstack((r, g, b))
    # resize like in training
    img = resize(img, (512, 512))
    # get inference
    pred = infer(img, model, inf_aug)
    # ensure right shape
    assert pred.shape[0]==1
    assert pred.shape[1]==19
    return(pred[0])


def convert_sample_submission(path='/kaggle/input/hpa-single-cell-image-classification/sample_submission.csv')-> pd.core.frame.DataFrame:
    """
    Input: path to sample_submission.csv
    Output: csv submission without PredictionString
    """
    # read sample_submission csv
    sample_submission = pd.read_csv(path)
    # create new dataframe without prediction String
    submission = sample_submission.drop(columns = ['PredictionString'])
    return(submission)
    
    
def read_image_array(ID, path='/kaggle/input/hpa-single-cell-image-classification/test/'):
    """
    Input: ID and path to images
    Output: np arrays for each channels
    """
    r = imageio.imread(path+ID+'_red.png')
    g = imageio.imread(path+ID+'_green.png')
    b = imageio.imread(path+ID+'_blue.png')
    y = imageio.imread(path+ID+'_yellow.png')
    return r, g, b, y


def segment(ref_channels: List[str], segmentator: CellSegmentator)-> np.ndarray:
    """
    Input: reference channels for segementation class and segmentation class
    Output: mask for image
    """
    # Ref channels must be in order red, yellow, blue
    input_ = [[i] for i in ref_channels]  # Segmentator only accepts list of lists input
    nuc_segmentation = segmentator.pred_nuclei(input_[2])[0]
    cell_segmentation = segmentator.pred_cells(input_)[0]
    mask = label_cell(nuc_segmentation, cell_segmentation)[1]
    return mask


def get_rle_string(rgby_channel: tuple, binary_mask: np.ndarray, model, inf_aug)-> str:
    """
    Input: cell rgby channels and binary mask for cell
    Pass: pass on model and augmentations for inference
    Output: prediction string for cell
    """
    # get predictions
    class_preds = get_inference(rgby_channel, model, inf_aug)
    # add to prediction string
    enc = encode_binary_mask(binary_mask)
    # pred string
    pred_string = ''
    for jj, pred in enumerate(class_preds):
        pred_string += str(jj)+' '+str(pred)+' '+enc+ ' '
    return(pred_string)


def extract_and_predict(channels: tuple, mask: np.ndarray, model, inf_aug) -> str:
    """
    Input: whole image channels and mask for whole image
    Pass: passing on model and augmentations for inference
    Output: prediction string for whole image to be inputted to csv
    """
    # master predicted_string
    predicted_string = ''
    for label in np.unique(mask):
        # Get values from where image == label
        if label == 0:
            continue  # ignore background
        temp_mask = mask.copy()
        temp_mask[temp_mask != label] = 0
        temp_mask[temp_mask == label] = 1
        # Get temp mask bounding box coords
        idxs = np.asarray(temp_mask == 1).nonzero()
        y_min, y_max = idxs[0].min(), idxs[0].max()
        x_min, x_max = idxs[1].min(), idxs[1].max()

        # tuple of all channels to be classified
        rgby_channel_list = list()
        for channel in channels:
            # Zero pad and square off
            single_cell = temp_mask * channel
            single_cell = single_cell[y_min:(y_max + 1), x_min:(x_max + 1)]
            # append channel to list
            rgby_channel_list.append(single_cell)
        # return prediction string to append
        predicted_string += get_rle_string(tuple(rgby_channel_list), np.array(temp_mask, dtype=bool),
                                           model, inf_aug)
    return(predicted_string)


def encode_binary_mask(mask: np.ndarray)-> str:
    """
    Input: binary mask
    Output: encoded rle mask, decoded into ascii text
    """

    # check input mask --
    if mask.dtype != np.bool:
        raise ValueError(
           "encode_binary_mask expects a binary mask, received dtype == %s" %
           mask.dtype)

    mask = np.squeeze(mask)
    if len(mask.shape) != 2:
        raise ValueError(
           "encode_binary_mask expects a 2d mask, received shape == %s" %
           mask.shape)

    # convert input mask to expected COCO API input --
    mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
    mask_to_encode = mask_to_encode.astype(np.uint8)
    mask_to_encode = np.asfortranarray(mask_to_encode)

    # RLE encode mask --
    encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

    # compress and base64 encoding --
    binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
    base64_str = base64.b64encode(binary_str)
    return base64_str.decode('ascii')

# Inference

In [15]:
# load data
df = convert_sample_submission()
# load pretrained model
resnet_model = resnet18(pretrained=False)
resnet_model.load_state_dict(torch.load(RESNET_PATH))
# define model class
model = ResNet18(resnet_model)
# load weights
model.load(MODEL_PATH)
# augmentations
inf_aug = A.Compose([
A.Normalize(
    mean=MEAN_CHANNEL_VALUES, 
    std=CHANNEL_STD_DEV, 
    max_pixel_value=1.0,
    p=1.0
    )
])   

In [ ]:
# Iterate over test dataframce
for image in df.itertuples():
    print('#####################################################')
    print('Segmenting image {}'.format(image.ID))
    try:    
        # segment and extract
        r, g, b, y = read_image_array(image.ID)
        # assign ref channels
        ref_channels = [r, y, b]
        # get mask
        mask = segment(ref_channels, segmentator)
        # get predicted string
        df.at[image.Index, 'PredictedString'] = extract_and_predict((r, g, b, y), 
                                                                    mask,
                                                                    model,
                                                                    inf_aug)
    except:
        continue

#     # plot 
#     fig, ax = plt.subplots(figsize=(8, 8))
#     ax.imshow(np.dstack((r, g, b)))
#     ax.imshow(mask, alpha=0.3)
#     plt.title('Image Example: {}'.format(ID))
#     plt.show()
#     print(predicted_string)

#####################################################
Segmenting image 0040581b-f1f2-4fbe-b043-b6bfea5404bb
#####################################################
Segmenting image 004a270d-34a2-4d60-bbe4-365fca868193
#####################################################
Segmenting image 00537262-883c-4b37-a3a1-a4931b6faea5
#####################################################
Segmenting image 00c9a1c9-2f06-476f-8b0d-6d01032874a2
#####################################################
Segmenting image 0173029a-161d-40ef-af28-2342915b22fb
#####################################################
Segmenting image 01a14326-67b8-43b0-ac7a-ba6dfb3c38ad
#####################################################
Segmenting image 020a29cf-2c24-478b-8603-c22a90dc3e31
#####################################################
Segmenting image 02425037-3048-4ff3-9c0a-e9fc2d033e32
#####################################################
Segmenting image 02531b54-078d-4a33-9c9c-0632b58c0a9a
############################

In [ ]:
# how many images were unsucessful
df[df.PredictedString.isnull()==True]

In [ ]:
# submission cv
df.to_csv('/kaggle/working/submission.csv', index=False)